# read_data

> This reads the marriage patterns for the non-reform states in the 1970 wave of the Choo and Siow *JPE* 2006 data.

In [ ]:
#| default_exp read_data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import numpy as np
from pathlib import Path

In [ ]:
#| export
data_dir = Path.cwd().parent / "matching_separable_simuls" / "ChooSiow70nNdata"

def read_margins(
    ) -> tuple[np.ndarray, np.ndarray]: 
    """reads and returns the margins for men and for women """
    nx = np.loadtxt(data_dir / "nx70n.txt")
    my = np.loadtxt(data_dir / "my70n.txt")
    return nx, my

def read_marriages(
    ) -> tuple[np.ndarray, np.ndarray]: 
    """ reads and returns the marriages and the variances"""
    muxy = np.loadtxt(data_dir / "muxy70nN.txt")
    varmus = np.loadtxt(data_dir / "varmus70nN.txt")
    return muxy, varmus

nx, my = read_margins()
muxy, varmus = read_marriages()
    

    
    

In [ ]:
#| hide
test_eq(muxy[3,4], 16730.0)
test_eq(nx[-1], 84399)
test_eq(my[7], 281192)
test_eq(varmus.shape, (675, 675))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()